In [1]:
import nltk

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
 
# dl = nltk.downloader.Downloader('http://nltk.org/nltk_data/index.xml')
# # nltk.download('stopwords')
# dl.download('stopwords')

from nltk.corpus import stopwords
stopwords = stopwords.words('english')    
#print(stopwords) # see default stopwords

def stop_word_removal(row):
    token = [token for token in row if token not in stopwords]
    token = filter(None, token)
    return token

In [2]:
# Params for bert model and tokenization
Nsamp = 200 # number of samples to generate in each class - 'spam', 'not spam'
maxtokens = 200 # the maximum number of tokens per document
maxtokenlen = 100 # the maximum length of each token
Enron_Samp = 1000

In [3]:
def reg_expressions(row):
    tokens = []
    try:
        for token in row:
            token = token.lower()
            token = re.sub(r'[\W\d]', "", token)
            token = token[:maxtokenlen] # truncate token
            tokens.append(token)
    except:
        token = ""
        tokens.append(token)
    return tokens

In [4]:
def tokenize(row):
    if row is None or row is '':
        tokens = ""
    else:
        try:
            tokens = row.split(" ")[:maxtokens]
        except:
            tokens=""
    return tokens

In [5]:
import re, string

def clean_text(text):
    text = text.encode('utf-8').decode('utf-8')
    while '\n' in text:
        text = text.replace('\n', ' ')
    while '  ' in text:
        text = text.replace('  ', ' ')
    words = text.split()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = []
    for token in words: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            stripped.append(new_token.lower())
    text = ' '.join(stripped)
    return text

#jose data

In [6]:
import os
jose_phishing_files = []

for dirname, _, filenames in os.walk('../data/jose_phishing_html'):
    for filename in filenames:
        if filename.startswith('phishing'):
            jose_phishing_files.append(os.path.join(dirname, filename))
jose_phishing_files[0]    

'../data/jose_phishing_html/phishing5.mbox-1805.eml.html'

In [7]:
def get_data_content(files):
    data = []
    for file in files:
        with open(file, 'r') as f:
            data.append(f.read())
    return data
    
jose_file_contents = get_data_content(jose_phishing_files)
len(jose_file_contents)

10147

In [8]:
import pandas as pd
jose_bodies_df = pd.DataFrame(jose_file_contents[:Nsamp],columns=["message"],dtype=str)
# jose_bodies_df = pd.DataFrame(jose_bodies_df[1:])

jose_bodies_df.head() # yo

,message
0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
1,"<html dir=""ltr"">\n<head>\n<meta http-equiv=""Co..."
2,"<TABLE cellSpacing=0 cellPadding=0 width=""100%..."
3,Your mailbox quota has been exceeded the stora...
4,"<style type=""text/css"">\n.n {\n\tcolor: #FF000..."


In [9]:
len(jose_bodies_df)

200

#Enron email data.

In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
filepath = "../data/enron/emails.csv"

# Read the enron data into a pandas.DataFrame called emails
emails = pd.read_csv(filepath)

print("Successfully loaded {} rows and {} columns!".format(emails.shape[0], emails.shape[1]))
emails.head()

Successfully loaded 517401 rows and 2 columns!


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [11]:
#emails.loc[100]["message"]


In [12]:
import email

def extract_messages(df):
    messages = []
    for item in df["message"]:
        # Return a message object structure from a string
        e = email.message_from_string(item)    
        # get message body  
        message_body = e.get_payload()
        messages.append(message_body)
    print("Successfully retrieved message body from e-mails!")
    return messages

bodies = extract_messages(emails)

Successfully retrieved message body from e-mails!


In [13]:
len(bodies)

517401

In [14]:
import random
bodies_df = pd.DataFrame(random.sample(bodies, Enron_Samp))

# expand default pandas display options to make emails more clearly visible when printed
pd.set_option('display.max_colwidth', 300)

bodies_df.head() # 

,0
0,"Judi,\n\nDid you receive a quote for Williams yet? When I spoke with Terri she said that $5000 USD was a pretty good estimate for my move.\n\nChris\n\n -----Original Message-----\nFrom: \t""Judi During"" <JDuring@royallepage2.com>@ENRON [mailto:IMCEANOTES-+22Judi+20During+22+20+3CJDuring+40royalle..."
1,"Thanks.\n\n -----Original Message-----\nFrom: \tVos, Theresa \nSent:\tWednesday, June 13, 2001 1:46 PM\nTo:\tPresto, Kevin M.; Zisman, Stuart\nCc:\tWhite, Stacey; Vinson, Donald Wayne; Pierce, Jody; Schield, Elaine\nSubject:\tRE: New Albany\n\nElaine Schield, ENA Accounting, agrees that since 8..."
2,"Yes, she'snot familiar with the music, but said she would listen to the music \nand see what she thought, but I have put in a call to my other friend, but I \nknow she would do it if her schedule was open, but she does is \nprofessionally, and I know there would be a charge, I should hear somet..."
3,"Just in case I forget to mention it, I need to move our SF visit to Friday \nam, due to travel plans. Is this still ok with you?\n\nKay"
4,"Please, remove me from the list.\n\n\n\n\n\n\n\nspawn@aclnet.cz on 12/09/2000 06:31:36 AM\nPlease respond to adamsd34@hongkong.com\nTo: <joe454@hongkong.com>\ncc: \nSubject: Are you looking for a ""better way"" to make money? 5400\n\n\n\n\n\n\nDo You Have The Yen To Be a A Millionaire?\n\n100% re..."


#fraudulent corpus

In [15]:
filepath = "../data/fradulent_emails.txt"
with open(filepath, 'r',encoding="latin1") as file:
    data = file.read()
    
# split on a code word appearing close to the beginning of each email
fraud_emails = data.split("From r")
len(fraud_emails)

3978

In [16]:
#fraud_emails[1]


In [17]:
fraud_bodies = extract_messages(pd.DataFrame(fraud_emails[:Nsamp + 1],columns=["message"],dtype=str))
fraud_bodies_df = pd.DataFrame(fraud_bodies[1:])

fraud_bodies_df.head() # yo

Successfully retrieved message body from e-mails!


,0
0,"FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-27-587908.\nE-MAIL: (james_ngola2002@maktoob.com).\n\nURGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\n\nDEAR FRIEND,\n\nI AM ( DR.) JAMES NGOLA, THE PERSONAL ASSISTANCE TO THE LATE CONGOLESE (PRESIDENT LAURENT KABILA) WHO WAS ASSASSINATED BY HIS BODY G..."
1,"Dear Friend,\n\nI am Mr. Ben Suleman a custom officer and work as Assistant controller of the Customs and Excise department Of the Federal Ministry of Internal Affairs stationed at the Murtala Mohammed International Airport, Ikeja, Lagos-Nigeria.\n\nAfter the sudden death of the former Head of s..."
2,"FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ELEME KINGDOM \nCHIEF DANIEL ELEME, PHD, EZE 1 OF ELEME.E-MAIL \nADDRESS:obong_715@epatra.com \n\nATTENTION:PRESIDENT,CEO Sir/ Madam. \n\nThis letter might surprise you because we have met\nneither in person nor by correspondence. But I believe\nit is..."
3,"FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF ELEME KINGDOM \nCHIEF DANIEL ELEME, PHD, EZE 1 OF ELEME.E-MAIL \nADDRESS:obong_715@epatra.com \n\nATTENTION:PRESIDENT,CEO Sir/ Madam. \n\nThis letter might surprise you because we have met\nneither in person nor by correspondence. But I believe\nit is..."
4,"Dear sir, \n \nIt is with a heart full of hope that I write to seek your help in respect of the context below. I am Mrs. Maryam Abacha the former first lady of the former Military Head of State of Nigeria General Sani Abacha whose sudden death occurred on 8th of June 1998 as a result of cardiac ..."


#IWSPA


In [18]:
# paths of the input files
no_head_train_path_0 = '../data/phishing/IWSPA-AP-traindata/phish/'
no_head_train_path_1 = '../data/phishing/IWSPA-AP-traindata/legit/'
head_train_path_0 = '../data/phishing/Dataset_Full_Header_Training/Dataset_Submit_Phish/'
head_train_path_1 = '../data/phishing/Dataset_Full_Header_Training/Dataset_Submit_Legit/'
no_head_test_path = '../data/phishing/IWSPA-APTestData/testdata_noheaders/'
head_test_path = '../data/phishing/IWSPA-APTestData/testdata_fullheaders/'

In [19]:
import os, re, string
import numpy as np

In [20]:
def clean_text(text):
    text = text.encode('utf-8').decode('utf-8')
    while '\n' in text:
        text = text.replace('\n', ' ')
    while '  ' in text:
        text = text.replace('  ', ' ')
    words = text.split()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = []
    for token in words: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            stripped.append(new_token.lower())
    text = ' '.join(stripped)
    return text

In [21]:

def get_data(path):
    text_list = list()
    files = os.listdir(path)
    for text_file in files:
        file_path = os.path.join(path, text_file)
        read_file = open(file_path,'r+')
        read_text = read_file.read()
        read_file.close()
        cleaned_text = clean_text(read_text)
        text_list.append(cleaned_text)
    return text_list, files

In [22]:
no_head_train_phish, temp = get_data(no_head_train_path_0)
no_head_train_legit, temp = get_data(no_head_train_path_1)
#head_train_0, temp = get_data(head_train_path_0)
#head_train_1, temp = get_data(head_train_path_1)
no_head_test, no_head_files = get_data(no_head_test_path)
#head_test, head_files = get_data(head_test_path)

In [23]:
# no_head_train = no_head_train_phish + no_head_train_legit
# no_head_labels_train = ([0] * len(no_head_train_phish)) + ([1] * len(no_head_train_legit))

#head_train = head_train_0 + head_train_1
#head_labels_train = ([0] * len(head_train_0)) + ([1] * len(head_train_1))

In [24]:
import pandas as pd
aw_legit_df = pd.DataFrame(no_head_train_legit[:Nsamp],columns=["message"],dtype=str)
aw_phish_df = pd.DataFrame(no_head_train_phish[:Nsamp],columns=["message"],dtype=str)
# jose_bodies_df = pd.DataFrame(jose_bodies_df[1:])

aw_legit_df.head() 

,message
0,hello everyone just wanted to see if we can reach a decision here thank you
1,gareth it seems to me from reading your email that you do not realize that this is precisely what magiccscengen already does ie it uses the scaling method that ben santer and i invented in the late 1980s to get time dependent patterns of future climate change i am attaching a description of the ...
2,daniele milan updated a event in the tentative activity calendar jun 09 2014 until jun 13 2014 knb advanced trainingyou are receiving this notification because you are watching the following tentative activity sent by atlassian confluence 544 the enterprise wiki
3,thank you 20 jordan kaplan national finance director democratic national committee 202 4885002 o 312 3390224 c userdomaincom
4,does stuart make many mistakes sammi fuck i hate her


In [25]:
aw_legit_df.head() 

,message
0,hello everyone just wanted to see if we can reach a decision here thank you
1,gareth it seems to me from reading your email that you do not realize that this is precisely what magiccscengen already does ie it uses the scaling method that ben santer and i invented in the late 1980s to get time dependent patterns of future climate change i am attaching a description of the ...
2,daniele milan updated a event in the tentative activity calendar jun 09 2014 until jun 13 2014 knb advanced trainingyou are receiving this notification because you are watching the following tentative activity sent by atlassian confluence 544 the enterprise wiki
3,thank you 20 jordan kaplan national finance director democratic national committee 202 4885002 o 312 3390224 c userdomaincom
4,does stuart make many mistakes sammi fuck i hate her


In [26]:
# legit
EnronEmails = bodies_df.iloc[:,0]#.apply(tokenize)
# EnronEmails = bodies_df.iloc[:,0].apply(tokenize)
# EnronEmails = EnronEmails.apply(stop_word_removal)
# EnronEmails = EnronEmails.apply(reg_expressions)
# EnronEmails = EnronEmails.sample(len(EnronEmails))

In [27]:
# iwspa
IwspaEmails_legit =  aw_legit_df.iloc[:, 0]#.apply(tokenize)
# IwspaEmails_legit = IwspaEmails_legit.sample(len(IwspaEmails_legit))

In [28]:
# phishing
fraudEmails = fraud_bodies_df.iloc[:,0]#.apply(tokenize)
# fraudEmails = fraud_bodies_df.iloc[:,0].apply(tokenize)
# fraudEmails = fraudEmails.apply(stop_word_removal)
# fraudEmails = fraudEmails.apply(reg_expressions)
# fraudEmails = fraudEmails.sample(len(fraudEmails))

In [29]:
# iwspa
IwspaEmails_phish =  aw_phish_df.iloc[:, 0]#.apply(tokenize)
# IwspaEmails_phish = IwspaEmails_phish.sample(len(IwspaEmails_phish))

In [30]:
# jose
JoseEmails = jose_bodies_df.iloc[:, 0]#.apply(tokenize)
# JoseEmails = JoseEmails.sample(len(JoseEmails))


raw_data = pd.concat([fraudEmails, IwspaEmails_phish, JoseEmails, EnronEmails, IwspaEmails_legit], axis=0).values


In [31]:
len(raw_data)

1800

In [32]:
# 1:spam 0:not spam
print("Shape of combined data represented as numpy array is:")
print(raw_data.shape)
#print("Data represented as numpy array is:")
#print(raw_data)

# corresponding labels
Categories = ['spam','notspam']
#header = ([1]*Nsamp)
#header.extend(([0]*Nsamp))
header = ([1]*(len(fraudEmails) + len(IwspaEmails_phish) + len(JoseEmails)))
header.extend(([0]*(len(EnronEmails) + len(IwspaEmails_legit))))

Shape of combined data represented as numpy array is:
(1800,)


In [33]:
type(raw_data[10])

str

In [34]:
# function for shuffling data in unison with labels/header
def unison_shuffle(a, b):
    p = np.random.permutation(len(b))
    data = a[p]
    header = np.asarray(b)[p]
    return data, header

# function for converting data into the right format, due to the difference in required format from sklearn models
# we expect a single string per email here, versus a list of tokens for the sklearn models previously explored
def convert_data(raw_data,header):
    converted_data, labels = [], []
    for i in range(raw_data.shape[0]):
        # out = ' '.join(str(raw_data[i]))
        out = str(raw_data[i])
        converted_data.append(out)
        labels.append(header[i])
        #print(i)
    converted_data = np.array(converted_data, dtype=object)[:, np.newaxis]
    
    return converted_data, np.array(labels)

raw_data, header = unison_shuffle(raw_data, header)

# split into independent 70% training and 30% testing sets
idx = int(0.7*raw_data.shape[0])
# 70% of data for training
train_x, train_y = convert_data(raw_data[:idx],header[:idx])
# remaining 30% for testing
test_x, test_y = convert_data(raw_data[idx:],header[idx:])

print("train_x/train_y list details, to make sure it is of the right form:")
print(len(train_x))
# print(train_x)
# print(train_y[:5])
print(train_y.shape)


train_x/train_y list details, to make sure it is of the right form:
1260
(1260,)


In [35]:
np.save('../output/train_x.npy', train_x)
np.save('../output/train_y.npy', train_y)
np.save('../output/test_x.npy', test_x)
np.save('../output/test_y.npy', test_y)

In [36]:
b = np.load('../output/train_x.npy', allow_pickle=True)

issue = np.array_equal(train_x,b)  #验证  比较a，b是否相同时同一个array
print(str(issue))

b = np.load('../output/train_y.npy', allow_pickle=True)

issue = np.array_equal(train_y,b)
print(str(issue))

b = np.load('../output/test_x.npy', allow_pickle=True)

issue = np.array_equal(test_x,b)  #验证  比较a，b是否相同时同一个array
print(str(issue))

b = np.load('../output/test_y.npy', allow_pickle=True)

issue = np.array_equal(test_y,b)
print(str(issue))

True
True
True
True


In [37]:
store = pd.concat([pd.DataFrame(train_x), pd.DataFrame(train_y)], axis=1).values
store1 = pd.concat([pd.DataFrame(test_x), pd.DataFrame(test_y)], axis=1).values

In [38]:
store[0]


array(['F R O M : M R .   J A M E S   N G O L A . \n C O N F I D E N T I A L   T E L :   2 3 3 - 2 7 - 5 8 7 9 0 8 . \n E - M A I L :   ( j a m e s _ n g o l a 2 0 0 2 @ m a k t o o b . c o m ) . \n \n U R G E N T   B U S I N E S S   A S S I S T A N C E   A N D   P A R T N E R S H I P . \n \n \n D E A R   F R I E N D , \n \n I   A M   (   D R . )   J A M E S   N G O L A ,   T H E   P E R S O N A L   A S S I S T A N C E   T O   T H E   L A T E   C O N G O L E S E   ( P R E S I D E N T   L A U R E N T   K A B I L A )   W H O   W A S   A S S A S S I N A T E D   B Y   H I S   B O D Y   G U A R D   O N   1 6 T H   J A N .   2 0 0 1 . \n \n \n T H E   I N C I D E N T   O C C U R R E D   I N   O U R   P R E S E N C E   W H I L E   W E   W E R E   H O L D I N G   M E E T I N G   W I T H   H I S   E X C E L L E N C Y   O V E R   T H E   F I N A N C I A L   R E T U R N S   F R O M   T H E   D I A M O N D   S A L E S   I N   T H E   A R E A S   C O N T R O L L E D   B Y   ( D . R . C . )   D E M 

In [39]:
csv = pd.DataFrame(store)
csv.to_csv('../output/train_1.csv')

In [40]:
csv = pd.DataFrame(store1)
csv.to_csv('../output/train_2.csv')

In [41]:
train_1 = pd.read_csv('../output/train_1.csv')
train_1.head()

,Unnamed: 0,0,1
0,0,F R O M : M R . J A M E S N G O L A . \n C O N F I D E N T I A L T E L : 2 3 3 - 2 7 - 5 8 7 9 0 8 . \n E - M A I L : ( j a m e s _ n g o l a 2 0 0 2 @ m a k t o o b . c o m ) . \n \n U R G E N T B U S I N E S S A S S I S T A N C E A N D P A R T N E R S H I P . \n \n \n D E A R...,1
1,1,t o b e a u t o m a t i c a l l y u n s u b s c r i b e d f r o m t h i s l i s t p l e a s e e m a i l u s e r d o m a i n c o m,0
2,2,d a n i e l e m i l a n u p d a t e d a e v e n t i n t h e c o n f e r e n c e s m a r k e t i n g e v e n t s c a l e n d a r j u n 1 7 2 0 1 4 u n t i l j u n 1 9 2 0 1 4 i d e c r o m a l i n k y o u a r e r e c e i v i n g t h i s n o t i f ...,0
3,3,"< D I V s t y l e = "" b a c k g r o u n d : u r l ( ' h t t p : / / w w w . c a r t a s i . i t / I m m a g i n i / b k g _ s e c o n d _ l e v e l . g i f ' ) ; w i d t h : 1 0 0 % ; h e i g h t : 3 5 p x ; "" > \n \t < a s t y l e = "" p a d d i n g : 6 p x 0 0 3 0 p x ; ...",1
4,4,r e a d o n t h e w e b l i n k o n p a r e n t i n g 2 0 t e e n s s a y t h e y 8 2 1 7 r e a d d i c t e d t o t e c h n o l o g y h e r e 8 2 1 7 s h o w p a r e n t s c a n h e l p h a l f o f a l l t e e n s s a y t h e y f e e l a d d ...,0


In [42]:
train_2 = pd.read_csv('../output/train_2.csv')
train_2.head()

,Unnamed: 0,0,1
0,0,S a r a h - \n \n A t t a c h e d a r e m y c o m m e n t s t o t h e p r o p o s e d N o n - D i s c l o s u r e A g r e e m e n t . P l e a s e \n r e v i e w t h e m a t y o u r c o n v e n i e n c e a n d l e t m e k n o w y o u r t h o u g h t s ....,0
1,1,"i f t h e y a r e n o t c o n f i r m i n g t h i s d e a l a r e t h e y c o n f i r m i n g 1 0 3 4 2 5 4 ? \n \n - - - - - O r i g i n a l M e s s a g e - - - - - \n F r o m : \t D o z i e r , S c o t t \n S e n t : \t T u e s d a y , O c t o b e r 2 3 , ...",0
2,2,- - - - - - - - - - - - - - - - - - - - - - F o r w a r d e d b y K a y M a n n / C o r p / E n r o n o n 1 2 / 1 9 / 2 0 0 0 1 1 : 5 5 \n A M - - - - - - - - - - - - - - - - - - - - - - - - - - - \n \n \n K a y M a n n \n 1 2 / 1 5 / 2 0 0 0 0 1 : 2 2 P M \n T o : L y ...,0
3,3,A t t a c h e d i s a n A s s i g n e d C o m m i s s i o n e r ' s R u l i n g r e a c c e s s t o D W R ' s c o m p u t e r \n m o d e l s u s e d t o d e t e r m i n e i t s r e v e n u e r e q u i r e m e n t . I t m a y b e a f o r e s h a d o w i...,0
4,4,Y o u r u n d e r s t a n d i n g o f m y p r e v i o u s d i s s e r t a t i o n i s c o r r e c t . - J i m \n \n \n F r o m : J e f f D a s o v i c h @ E N R O N o n 0 2 / 2 8 / 2 0 0 1 0 1 : 0 4 P M C S T \n S e n t b y : J e f f D a s o v i c h @ E N R ...,0


#enron html data

In [43]:
# def extract_html_messages(df):
#     messages = []
#     for item in df["message"]:
#         # Return a message object structure from a string
#         e = email.message_from_string(item)
#         # get message body
#         message_body = e.get_payload()
#         if '<html>' in message_body:
#             messages.append(message_body)
#         # if BeautifulSoup(message_body, "html.parser").find():
#         #     messages.append(message_body)
#     print("Successfully retrieved message body from e-mails!")
#     print(len(messages))
#     return messages
# enron_html_bodies = extract_html_messages(emails)

In [44]:
# enron_html_bodies[0]


#positive: legit
#enron text:50W html:874
#iwspa text:5091 html:9

#negative: phishing
#fraudulent text:3978 html:147
#iwspa text:628 html:15
#jose text:10147 html:10147



#dataset:
#fradulent all 
#jose all html
#iwspa all no header
#enron 2W text + all html